Use cases of Axon python API
============================

Load Brainvisa
--------------

The following lines enable to load Brainvisa without the graphical user interface through a python script:

In [ ]:
import brainvisa.axon
brainvisa.axon.initializeProcesses()

Call a process
--------------

The module [brainvisa.processes](python_bv_processing.html#module-brainvisa.processes) manages the processes, pipelines and their execution.

The object [brainvisa.processes.ExecutionContext](python_bv_processing.html#brainvisa.processes.ExecutionContext) enables to start a Brainvisa process using the method [brainvisa.processes.ExecutionContext.runProcess](python_bv_processing.html#brainvisa.processes.ExecutionContext.runProcess).
When Brainvisa is loaded, a default execution context exists and is returned by the function [brainvisa.processes.defaultContext](python_bv_processing.html#brainvisa.processes.defaultContext)

The following example will run the process Threshold on an image:


In [ ]:
import brainvisa.processes
context = brainvisa.processes.defaultContext()

# Just to get an example data for running the process
from brainvisa.configuration.neuroConfig import getSharePath, bvShareDirectory
import os
example_data = os.path.join(getSharePath(), bvShareDirectory, "anatomical_templates", "MNI152_T1_1mm_brain_mask.nii")
# creating a temporary file for the output
output_data = context.temporary("NIFTI-1 image")

context.runProcess("threshold", image_input=example_data, image_output=output_data, threshold1=100)

Select a step in a pipeline
---------------------------

In a pipeline, some steps may be optional and can be selected or unselected for execution. 
It is possible to select or unselect a step of a pipeline before running it through a python script. 
A pipeline is a process that have execution nodes. The method [brainvisa.processes.Process.executionNode](python_bv_processing.html#brainvisa.processes.Process.executionNode) returns an [brainvisa.processes.ExecutionNode](python_bv_processing.html#brainvisa.processes.ExecutionNode).
The execution node of the pipeline contains child nodes, the name of these nodes can be obtained with the method [brainvisa.processes.ExecutionNode.childrenNames](python_bv_processing.html#brainvisa.processes.ExecutionNode.childrenNames).
To get a specific child node, the method [brainvisa.processes.ExecutionNode.child](python_bv_processing.html#brainvisa.processes.ExecutionNode.child) can be used.

The following examples gets an instance of the *convertOldDatabase* pipeline and unselects all its steps:


In [ ]:
pipeline=brainvisa.processes.getProcessInstance("convertOldDatabase")
nodes=pipeline.executionNode()

nodes.childrenNames()
assert(nodes.childrenNames() == ['ConvertDatabase', 'CheckDatabase', 'CleanDatabase'])
nodes.child("ConvertDatabase").setSelected(0)
assert(not nodes.child("ConvertDatabase").isSelected())
nodes.child("CheckDatabase").setSelected(0)
nodes.child("CleanDatabase").setSelected(0)

The process instance can be given as a parameter for the [brainvisa.processes.ExecutionContext.runProcess](python_bv_processing.html#brainvisa.processes.ExecutionContext.runProcess) method instead of the process id.


Query a database
----------------

At Brainvisa startup, an internal database and the database selected in the user's preferences are loaded. 
The list of databases ([brainvisa.data.sqlFSODatabase.SQLDatabases](python_bv_data.html#brainvisa.data.sqlFSODatabase.SQLDatabases)) is stored in the global variable [brainvisa.data.neuroHierarchy.databases](python_bv_data.html#brainvisa.data.neuroHierarchy.databases).
Each database is an instance of the class [brainvisa.data.sqlFSODatabase.SQLDatabase](python_bv_data.html#brainvisa.data.sqlFSODatabase.SQLDatabases).
Several methods enable to query a database or a list of databases. The results of queries are generally [brainvisa.data.neuroDiskItems.DiskItem](python_bv_data.html#brainvisa.data.neuroDiskItems.DiskItem) objects. A DiskItem represents data stored in files and indexed in a database with additionnal information.

In the following example, a DiskItem is searched in the databases by filename:

In [ ]:
from brainvisa.data.neuroHierarchy import databases
from brainvisa.data.neuroDiskItems import DiskItem

item = databases.getDiskItemFromFileName(example_data)
assert(isinstance(item, DiskItem))
assert(str(item.type) == 'anatomical Mask Template')
assert(str(item.format) == 'NIFTI-1 image')

Here is a request for all DiskItems of type "anatomical Template" with the value of *Size* attribute being "1 mm":

In [ ]:
items = databases.findDiskItems({"_type" : "anatomical Template", "Size": "1 mm"})
assert(str(items).startswith('<generator object SQLDatabases.findDiskItems'))
model_filename = next(items).fileName()
assert(model_filename.startswith(os.path.join(getSharePath(), bvShareDirectory, "anatomical_templates")))

The object [brainvisa.data.readdiskitem.ReadDiskItem](python_bv_data.html#brainvisa.data.readdiskitem.ReadDiskItem) enables to search for an existing DiskItem in the databases using the method [ReadDiskItem.findValue](python_bv_data.html#brainvisa.data.readdiskitem.ReadDiskItem.findValue). If the request is not precise enought and several DiskItems match, the method returns *None*.

In [ ]:
from brainvisa.data.readdiskitem import ReadDiskItem
from brainvisa.configuration import neuroConfig
rd = ReadDiskItem("anatomical Template", "Aims readable volume formats")
template = rd.findValue({"Size" : "2 mm"})
assert(template is None)
template = rd.findValue({"Size" : "2 mm", "skull_stripped": "no", 
                         "_database": neuroConfig.dataPath[0].directory})
assert(template is not None)
assert(template.fileName().startswith(os.path.join(getSharePath(), bvShareDirectory, "anatomical_templates")))

The object [brainvisa.data.writediskitem.WriteDiskItem](python_bv_data.html#brainvisa.data.writediskitem.WriteDiskItem) enables to create new DiskItems to write output data according to Brainvisa hierarchy of directories.

In [ ]:
from brainvisa.data.writediskitem import WriteDiskItem
wd = WriteDiskItem("Raw T1 MRI", "NIFTI-1 image")
item = wd.findValue({"protocol" : 'test', "subject" : "mysubject"})
# nothe that item is not None only if one (and only one) user database has been setup.

Quit Brainvisa
--------------

The function [brainvisa.axon.processes.cleanup](python_bv_processing.html#brainvisa.axon.processes.cleanup) should be called at the end of the script to quit properly Brainvisa.

In [ ]:
brainvisa.axon.cleanup()